# Import

In [1]:
%install-swiftpm-flags -c release
%install '.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.3")' SwiftCoreMLTools

Installing packages:
	.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.3")
		SwiftCoreMLTools
With SwiftPM flags: ['-c', 'release']
Working in: /tmp/tmp7mi7zk4n/swift-install
Fetching https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git
Fetching https://github.com/apple/swift-protobuf.git
Cloning https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git
Resolving https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git at 0.0.3
Cloning https://github.com/apple/swift-protobuf.git
Resolving https://github.com/apple/swift-protobuf.git at 1.8.0
[1/2] Compiling SwiftProtobuf AnyMessageStorage.swift
[2/3] Compiling SwiftCoreMLTools Activations.swift
[3/4] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
[4/4] Linking libjupyterInstalledPackages.so
Initializing Swift...
Installation complete!


In [2]:
import Foundation
import TensorFlow
import SwiftCoreMLTools

# Data Ingestion

In [3]:
let trainCSV = try String(contentsOfFile:"../data/train.csv", encoding: String.Encoding.utf8)
let testCSV = try String(contentsOfFile:"../data/test.csv", encoding: String.Encoding.utf8)

let trainRecords: [[Float]] = trainCSV.split(separator: "\n").map{ String($0).split(separator: ",").compactMap{ Float(String($0)) } }
let testRecords: [[Float]] = testCSV.split(separator: "\n").map{ String($0).split(separator: ",").compactMap{ Float(String($0)) } }

let numTrainRecords = trainRecords.count
let numTrainColumns = trainRecords[0].count
let numTestRecords = testRecords.count
let numTestColumns = testRecords[0].count

print(numTrainRecords, numTrainColumns, numTestRecords, numTestColumns)

let xTrain = trainRecords.map{ Array($0[0..<numTrainColumns-1]) }
let yTrain = trainRecords.map{ [$0[numTrainColumns-1]] }
let xTest = testRecords.map{ Array($0[0..<numTestColumns-1]) }
let yTest = testRecords.map{ [$0[numTestColumns-1]] }

print(xTrain.count, xTrain[0].count, yTrain.count, yTrain[0].count,
      xTest.count, xTest[0].count, yTest.count, yTest[0].count)

let xAllTrain = Array(xTrain.joined())
let yAllTrain = Array(yTrain.joined())
let xAllTest = Array(xTest.joined())
let yAllTest = Array(yTest.joined())

print(xAllTrain.count, yAllTrain.count, xAllTest.count, yAllTest.count)

405 14 101 14
405 13 405 1 101 13 101 1
5265 405 1313 101


In [4]:
let XTrain = Tensor<Float>(xAllTrain).reshaped(to: TensorShape([numTrainRecords, numTrainColumns-1]))
let YTrain = Tensor<Float>(yAllTrain).reshaped(to: TensorShape([numTrainRecords, 1]))
let XTest = Tensor<Float>(xAllTest).reshaped(to: TensorShape([numTestRecords, numTestColumns-1]))
let YTest = Tensor<Float>(yAllTest).reshaped(to: TensorShape([numTestRecords, 1]))

print(XTrain.shape, YTrain.shape, XTest.shape, YTest.shape)

[405, 13] [405, 1] [101, 13] [101, 1]


# Model

In [5]:
struct RegressionModel: Layer {
    var layer1 = Dense<Float>(inputSize: 13, outputSize: 64, activation: relu)
    var layer2 = Dense<Float>(inputSize: 64, outputSize: 32, activation: relu)
    var layer3 = Dense<Float>(inputSize: 32, outputSize: 1)
    
    @differentiable
    func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
        return input.sequenced(through: layer1, layer2, layer3)
    }
}

var model = RegressionModel()

# Training

In [6]:
let optimizer = RMSProp(for: model, learningRate: 0.001)
Context.local.learningPhase = .training

In [7]:
let epochCount = 500
let batchSize = 32
let numberOfBatch = Int(ceil(Double(xTrain.count) / Double(batchSize)))
let shuffle = true

func mae(predictions: Tensor<Float>, truths: Tensor<Float>) -> Float {
    return abs(Tensor<Float>(predictions - truths)).mean().scalarized()
}

In [8]:
for epoch in 1...epochCount {
    var epochLoss: Float = 0
    var epochMAE: Float = 0
    var batchCount: Int = 0
    var batchArray = Array(repeating: false, count: numberOfBatch)
    for batch in 0..<numberOfBatch {
        var r = batch
        if shuffle {
            while true {
                r = Int.random(in: 0..<numberOfBatch)
                if !batchArray[r] {
                    batchArray[r] = true
                    break
                }
            }
        }
        
        let batchStart = r * batchSize
        let batchEnd = min(xTrain.count, batchStart + batchSize)
        let (loss, grad) = model.valueWithGradient { (model: RegressionModel) -> Tensor<Float> in
            let logits = model(XTrain[batchStart..<batchEnd])
            return meanSquaredError(predicted: logits, expected: YTrain[batchStart..<batchEnd])
        }
        optimizer.update(&model, along: grad)
        
        let logits = model(XTrain[batchStart..<batchEnd])
        epochMAE += mae(predictions: logits, truths: YTrain[batchStart..<batchEnd])
        epochLoss += loss.scalarized()
        batchCount += 1
    }
    epochMAE /= Float(batchCount)
    epochLoss /= Float(batchCount)

    print("Epoch \(epoch): MSE: \(epochLoss), MAE: \(epochMAE)")
}

Epoch 1: MSE: 545.45825, MAE: 21.518887
Epoch 2: MSE: 470.22858, MAE: 19.819317
Epoch 3: MSE: 394.67313, MAE: 17.905602
Epoch 4: MSE: 314.3095, MAE: 15.6274805
Epoch 5: MSE: 232.63268, MAE: 13.046525
Epoch 6: MSE: 158.57167, MAE: 10.245337
Epoch 7: MSE: 105.07023, MAE: 7.9450307
Epoch 8: MSE: 71.832855, MAE: 6.453923
Epoch 9: MSE: 53.762688, MAE: 5.517841
Epoch 10: MSE: 42.892025, MAE: 4.8498416
Epoch 11: MSE: 35.520504, MAE: 4.349806
Epoch 12: MSE: 30.823189, MAE: 3.993562
Epoch 13: MSE: 27.556156, MAE: 3.7260425
Epoch 14: MSE: 25.184244, MAE: 3.5571535
Epoch 15: MSE: 23.511566, MAE: 3.4082675
Epoch 16: MSE: 22.121471, MAE: 3.2735124
Epoch 17: MSE: 20.755451, MAE: 3.14571
Epoch 18: MSE: 19.777336, MAE: 3.08034
Epoch 19: MSE: 18.853922, MAE: 2.9876635
Epoch 20: MSE: 18.073534, MAE: 2.9086761
Epoch 21: MSE: 17.422783, MAE: 2.8606844
Epoch 22: MSE: 16.74918, MAE: 2.7822561
Epoch 23: MSE: 16.224514, MAE: 2.7252994
Epoch 24: MSE: 15.769237, MAE: 2.7114346
Epoch 25: MSE: 15.408573, MAE: 2.6

Epoch 205: MSE: 4.7514243, MAE: 1.3903463
Epoch 206: MSE: 4.8268247, MAE: 1.3896642
Epoch 207: MSE: 4.7911463, MAE: 1.384557
Epoch 208: MSE: 4.762247, MAE: 1.3903694
Epoch 209: MSE: 4.753573, MAE: 1.3812392
Epoch 210: MSE: 4.8053594, MAE: 1.3743082
Epoch 211: MSE: 4.6894326, MAE: 1.3697855
Epoch 212: MSE: 4.587762, MAE: 1.3763392
Epoch 213: MSE: 4.6433535, MAE: 1.358465
Epoch 214: MSE: 4.6231966, MAE: 1.3436084
Epoch 215: MSE: 4.5627546, MAE: 1.3469517
Epoch 216: MSE: 4.5672274, MAE: 1.3527635
Epoch 217: MSE: 4.535068, MAE: 1.3519058
Epoch 218: MSE: 4.562803, MAE: 1.3406008
Epoch 219: MSE: 4.590995, MAE: 1.3474485
Epoch 220: MSE: 4.525178, MAE: 1.3426969
Epoch 221: MSE: 4.5432887, MAE: 1.3314558
Epoch 222: MSE: 4.5076137, MAE: 1.3192074
Epoch 223: MSE: 4.3467073, MAE: 1.3207386
Epoch 224: MSE: 4.5638566, MAE: 1.33726
Epoch 225: MSE: 4.4934864, MAE: 1.3180517
Epoch 226: MSE: 4.4525704, MAE: 1.3164482
Epoch 227: MSE: 4.50387, MAE: 1.3243153
Epoch 228: MSE: 4.4687138, MAE: 1.3293558
Epoch

Epoch 404: MSE: 2.6843185, MAE: 0.9982595
Epoch 405: MSE: 2.686189, MAE: 1.0018369
Epoch 406: MSE: 2.7149925, MAE: 0.9907176
Epoch 407: MSE: 2.7221334, MAE: 1.0003959
Epoch 408: MSE: 2.6590543, MAE: 0.9916578
Epoch 409: MSE: 2.5762126, MAE: 0.98055804
Epoch 410: MSE: 2.7350476, MAE: 0.99381006
Epoch 411: MSE: 2.6512516, MAE: 0.9915394
Epoch 412: MSE: 2.6154888, MAE: 0.9891421
Epoch 413: MSE: 2.710751, MAE: 0.9928638
Epoch 414: MSE: 2.527042, MAE: 0.9778876
Epoch 415: MSE: 2.5065618, MAE: 0.96751475
Epoch 416: MSE: 2.4994736, MAE: 0.9637321
Epoch 417: MSE: 2.6550076, MAE: 0.98084855
Epoch 418: MSE: 2.6384962, MAE: 0.97370523
Epoch 419: MSE: 2.6194904, MAE: 0.9733362
Epoch 420: MSE: 2.6240823, MAE: 0.9630117
Epoch 421: MSE: 2.5571897, MAE: 0.9751825
Epoch 422: MSE: 2.6235464, MAE: 0.9798405
Epoch 423: MSE: 2.580676, MAE: 0.9719228
Epoch 424: MSE: 2.5722668, MAE: 0.96672916
Epoch 425: MSE: 2.5864766, MAE: 0.96580535
Epoch 426: MSE: 2.5529747, MAE: 0.957828
Epoch 427: MSE: 2.5267944, MAE: 

# Test

In [19]:
Context.local.learningPhase = .inference
let cheat = model(XTrain)
print(cheat[0], YTrain[0])

let prediction = model(XTest)
print(prediction[0], YTest[0])

let tmse = meanSquaredError(predicted: prediction, expected: YTest).scalarized()
let tmae = mae(predictions: prediction, truths: YTest)

print("MSE: \(tmse), MAE: \(tmae)")

[22.837023] [22.6]
[20.796093] [18.9]
MSE: 5.5166893, MAE: 1.7715832


# Export

In [11]:
print(model.layer1.weight.shape, model.layer2.weight.shape, model.layer3.weight.shape)
print(model.layer1.bias.shape, model.layer2.bias.shape, model.layer3.bias.shape)

[13, 64] [64, 32] [32, 1]
[64] [32] [1]


In [9]:
let coremlModel = Model(version: 4,
                        shortDescription: "Regression",
                        author: "Jacopo Mangiavacchi",
                        license: "MIT",
                        userDefined: ["SwiftCoremltoolsVersion" : "0.0.3"]) {
    Input(name: "input", shape: [13], featureType: .Double)
    Output(name: "output", shape: [1], featureType: .Double)
    NeuralNetwork {
        InnerProduct(name: "dense1",
                     input: ["input"],
                     output: ["outDense1"],
                     weights: model.layer1.weight.flattened().scalars,
                     bias: model.layer1.bias.flattened().scalars,
                     inputChannels: 13,
                     outputChannels: 64,
                     updatable: false)
        ReLu(name: "Relu1",
             input: ["outDense1"],
             output: ["outRelu1"])
        InnerProduct(name: "dense2",
                     input: ["outRelu1"],
                     output: ["outDense2"],
                     weights: model.layer2.weight.flattened().scalars,
                     bias: model.layer2.bias.flattened().scalars,
                     inputChannels: 64,
                     outputChannels: 32,
                     updatable: false)
        ReLu(name: "Relu2",
             input: ["outDense2"],
             output: ["outRelu2"])
        InnerProduct(name: "dense3",
                     input: ["outRelu2"],
                     output: ["output"],
                     weights: model.layer3.weight.flattened().scalars,
                     bias: model.layer3.bias.flattened().scalars,
                     inputChannels: 32,
                     outputChannels: 1,
                     updatable: false)
    }
}

In [10]:
let coreMLData = coremlModel.coreMLData
try! coreMLData!.write(to: URL(fileURLWithPath: "../model/s4tf_train_model.mlmodel"))